In this notebook, the matroid encyclopedia is filled from scratch.
For each tuple k=(rank,size), and for each number mi in {0,...,#(matroids of type (rank,size))-1},
M[k][mi] is a list of properties of a matroid to which I refer in the thesis.

Run this file from top to bottom to fill the Matroid Encyclopedia in several minutes.
The function 'inspect' can be used to obtain the numbers or lists of matroids for each property; see below for examples.

In [121]:
#Edit the path to match your own directories
path = "/media/sf_SageShared/fromScratch/"

In [122]:
M = load(path + "matroidEncyclopediaEmpty_part1.sobj")
M2 = load(path + "matroidEncyclopediaEmpty_part2.sobj")
M.update(M2)
#loads the empty Matroid Encyclopedia, including minor relations and circuit-hyperplane relaxation relations.

In [123]:
#The following functions are used to fill up the matroid encyclopedia.

def kDual(k):
    return (k[1]-k[0],k[1])
def kDel(k):
    return (k[0],k[1]-1)
def kCon(k):
    return (k[0]-1,k[1]-1)

def addProperty(E,A,val,certificate=False):
    if A in E:
        if not E[A] == val:
            print "error!", k, mi, A, val, E[A]
    else:
        E[A] = val
        if certificate != False:
            E[A + " certificate"] = certificate

def AimpliesB(A,B):
    for k in M:
        for mi in M[k]:
            E = M[k][mi]
            if A in E and E[A]:
                addProperty(E,B,True,A)
            elif B in E and not E[B]:
                addProperty(E,A,False,B)

                
def closeDuality(A,certificate=True):
    for k in M:
        kd = kDual(k)
        for mi in M[k]:
            E = M[k][mi]
            if not A in E:
                continue
            di = E["dual"]
            Ed = M[kd][di]
            addProperty(Ed,A,E[A],("duality" if certificate else False))

def inheritFromMinor(A,val,principal=False,deletion=True,contraction=True,certificate=True):
    minorsr = "minors r"
    minorsr1 = "minors r-1"
    if principal:
        minorsr = "principal minors r"
        minorsr1 = "principal minors r-1"
    
    for n in range(10):
        for k in M:
            if k[1] != n:
                continue
            for mi in M[k]:
                E = M[k][mi]
                if A in E:
                    continue

                if deletion:
                    kd = kDel(k)
                    for i in E[minorsr]:
                        Ed = M[kd][i]
                        if A in Ed and Ed[A] == val:
                            addProperty(E,A,val,([minorsr,i] if certificate else False))
                            break

                    if A in E:
                        continue

                if contraction:            
                    kc = kCon(k)
                    for i in E[minorsr1]:
                        Ec = M[kc][i]
                        if A in Ec and Ec[A] == val:
                            addProperty(E,A,val,([minorsr1,i] if certificate else False))
                            break
                        
                        
def inheritFromExtension(A,val,principal=False,ext=True,coext=True,certificate=True):
    minorsr = "minors r"
    minorsr1 = "minors r-1"
    certr = "extension"
    certr1 = "coextension"
    if principal:
        minorsr = "principal minors r"
        minorsr1 = "principal minors r-1"
        certr = "principal extension"
        certr1 = "principal coextension"
    ns = range(10)
    ns.reverse()
    for n in ns:
        for k in M:
            if k[1] != n:
                continue
            for mi in M[k]:
                E = M[k][mi]
                if not (A in E and E[A] == val):
                    continue

                if ext:
                    kd = kDel(k)
                    for i in E[minorsr]:
                        Ed = M[kd][i]
                        addProperty(Ed,A,val,([certr,mi] if certificate else False) )

                if coext:            
                    kc = kCon(k)
                    for i in E[minorsr1]:
                        Ec = M[kc][i]
                        addProperty(Ec,A,val,([certr1,mi] if certificate else False))
                    
                    
def closeCHPrelaxation(A,val):
    for k in M:
        nChanged = 1
        while nChanged > 0:
            nChanged = 0
            for mi in M[k]:
                E = M[k][mi]
                if not (A in E and E[A] == val):
                    continue
                for i in E["circuit-hyperplane relaxations"]:
                    Er = M[k][i]
                    if not (A in Er):
                        nChanged += 1
                        addProperty(Er,A,val,["circuit-hyperplane relaxation", mi])

                    
def rank2Linear():
    for k in M:
        if k[0] <= 2 or k[1]-k[0] <= 2:
            for mi in M[k]:
                E = M[k][mi]
                addProperty(E,"2linear",True,"(co)rank at most 2")
                
                
def inspect(A,k,onlySize=True):
    values = {"-":[]}
    for mi in M[k]:
        E = M[k][mi]
        if not A in E:
            values["-"].append(mi)
        else:
            v = str(E[A])
            if not (v in values):
                values[v] = [mi]
            else:
                values[v].append(mi)
    if onlySize:
        return {v: len(values[v]) for v in values}
    else:
        return values            

In [124]:
#each matroid of (co)rank at most 2 is 2linear.
rank2Linear()

In [125]:
#a list of binary matroid indices for each (rank,size)
binary = load(path + "binaryMatroids.sobj")

#each binary matroid is 2linear.
for k in binary:
    for mi in binary[k]:
        addProperty(M[k][mi],"2linear",True,"binary")
del binary

In [126]:
#a list of 2linear matroid indices for (3,9)
lin93 = load(path + "linear2_39.sobj")
#the representations are found in "linear2representations93.sobj" as a list of [mi,representation];
#requires Sage 8.0 or higher

n = 9
for rep in lin93:
    r = rep[1]
    k = (r,n)
    addProperty(M[k][rep[0]],"2linear",True,"represented")
del lin93

In [127]:
#a list of 2linear matroid indices for (4,9)
lin94 = load(path + "linear2_49.sobj")
#the representations are found in "linear2representations94.sobj" as a list of [mi,representation];
#requires Sage 8.0 or higher

n = 9
for rep in lin94:
    r = rep[1]
    k = (r,n)
    addProperty(M[k][rep[0]],"2linear",True,"represented")
del lin94

In [128]:
#a list of nonlinear matroids due to a Groebner basis computation.
nonlin94 = load(path + "non2linear49_extra.sobj")

for mi in nonlin94:
    addProperty(M[k][mi],"2linear",False,"Groebner")
del nonlin94

In [129]:
#Linearity in characteristic 2 is closed under duality.
closeDuality("2linear")

In [130]:
#a list of [mi,representation] for rational matroids of type (3,9)
rat93 = load(path + "rationalRepresentations39.sobj")

n = 9
for rep in rat93:
    r = rep[1].nrows()
    k = (r,n)
    addProperty(M[k][rep[0]],"rational",True,"represented")
del rat93

In [131]:
#a list of [mi,representation] for rational matroids of type (4,9)
rat94 = load(path + "rationalRepresentations49.sobj")

n = 9
for rep in rat94:
    r = rep[1].nrows()
    k = (r,n)
    addProperty(M[k][rep[0]],"rational",True,"represented")
del rat94

In [132]:
#rationality is closed under duality
closeDuality("rational")

In [133]:
#a complete list of matroid indices per (rank,size) not satisfying the Dress-Wenzel condition
nonDW = load(path + "NonDressWenzel.sobj")

reg = nonDW["regular"]
for k in reg:
    for mi in reg[k]:
        addProperty(M[k][mi],"Dress-Wenzel condition",False)
    for mi in set([i for i in M[k]]) - set(reg[k]):
        addProperty(M[k][mi],"Dress-Wenzel condition",True)
    
#a complete list of matroid indices per (rank,size) not satisfying the Dress-Wenzel condition in characteristic 2
char2 = nonDW["char 2"]

for k in char2:
    for mi in char2[k]:
        addProperty(M[k][mi],"Dress-Wenzel condition char 2",False)
    for mi in set([i for i in M[k]]) - set(char2[k]):
        addProperty(M[k][mi],"Dress-Wenzel condition char 2",True)
del nonDW
del char2
del reg

In [134]:
#The Dress-Wenzel condition is closed under duality
closeDuality("Dress-Wenzel condition",False)

In [135]:
#The Dress-Wenzel condition in char 2 is closed under duality
closeDuality("Dress-Wenzel condition char 2",False)

In [136]:
#A list of matroid indices not satisfying the Ingleton-Main condition at depth d for each (rank,size)
nonIM = load(path + "NonIngletonMain.sobj")

for k in nonIM:
    for d in nonIM[k]:
        for mi in nonIM[k][d]:
            addProperty(M[k][mi],"Ingleton-Main condition",False)
            addProperty(M[k][mi],"Ingleton-Main condition depth",d)
del nonIM

In [137]:
#Not satisfying the Ingleton-Main condition is preserved under extension
inheritFromMinor("Ingleton-Main condition",False,certificate=False)

In [138]:
#We add some properties manually for small matroids.

addProperty(M[3,9][594],"2flock",False,"computed")
addProperty(M[6,9][594],"2flock",False,"computed")
#This is the ternary Reid geometry (and its dual)

addProperty(M[4,8][383],"2flock",False,"computed")
#This is the matroid T8.

addProperty(M[4,8][259],"2flock",False,"computed")

addProperty(M[3,9][560],"GF(4)(F)-representable",True,"represented (Lindstrom)")
#This is the Non-Pappus matroid

addProperty(M[3,9][655],"Quaternion representable",True,"represented")
#This is the matroid from Example 8.4

In [139]:
#A list of matroids represented over Q(sqrt(D)), where D = -1, -2 or -3.
#These are all representable over supersingular elliptic curves.
quat = load(path + "QuaternionRepresentable49.sobj")

for mi in quat:
    addProperty(M[4,9][mi],"Quaternion representable",True,"represented")
del quat

In [140]:
#A list of matroids represented over the rational Hamilton quaternions.
#These are all representable over a supersingular elliptic curve.
quat = load(path + "QuaternionSkewRepresentations49.sobj")

for mi in [f[0] for f in quat]:
    addProperty(M[4,9][mi],"Quaternion representable",True,"represented")
del quat

In [141]:
#skew-field representability is closed under duality
closeDuality("Quaternion representable")

In [142]:
#A list of matroids represented over Q(sqrt(D)), where D = -7 or -15.
#These are all representable over ordinary elliptic curves.
numb = load(path + "NumberFieldRepresentable49.sobj")

for mi in numb:
    addProperty(M[4,9][mi],"Quadratic imaginary number field representable",True,"represented")
del numb

In [143]:
#Representability over a commutative field is closed under duality
closeDuality("Quadratic imaginary number field representable")

In [144]:
#A list of matroids appearing as a deletion minor of Lindstroms 'grid' matroid.
grid = load(path + "LindstromGridMatroids49.sobj")

#All of these matroids are representable over GF(2)(F)
for mi in grid:
    addProperty(M[4,9][mi],"GF(4)(F)-representable",True,"represented (Lindstrom)")
del grid

In [145]:
#skew-field representability is closed under duality
closeDuality("Skew-field representable")

In [146]:
#a list of matroids for which Frobenius-flock representability was computed and True for each (rank,size).
flock = load(path + "FlockRepresentable.sobj")

for k in flock:
    for mi in flock[k]:
        addProperty(M[k][mi],"2flock",True,"computed")
del flock

In [147]:
#A list of matroids failing the Flock condition of type(4,9)
nonflock = load(path + "nonFlock49.sobj")

for mi in nonflock:
    addProperty(M[4,9][mi],"2flock",False,"computed")
    addProperty(M[5,9][mi],"2flock",False,"computed")
del nonflock

Now we apply known implications:

In [148]:
AimpliesB("2linear","Dress-Wenzel condition char 2")

In [149]:
AimpliesB("rational","Dress-Wenzel condition")

In [150]:
AimpliesB("Quadratic imaginary number field representable","Dress-Wenzel condition")

In [151]:
inheritFromExtension("2linear",True)
inheritFromExtension("rational",True)
inheritFromExtension("Quaternion representable",True)
inheritFromExtension("GF(4)(F)-representable",True)
inheritFromExtension("Quadratic imaginary number field representable",True)

In [152]:
inheritFromMinor("GF(4)(F)-representable",True,principal=True,contraction=False)
closeDuality("GF(4)(F)-representable")

In [153]:
AimpliesB("2linear","2algebraic")

In [154]:
AimpliesB("rational","2algebraic")

In [155]:
AimpliesB("Quaternion representable","2algebraic")

In [156]:
AimpliesB("Quadratic imaginary number field representable","2algebraic")

In [157]:
AimpliesB("GF(4)(F)-representable","2algebraic")

In [158]:
AimpliesB("2algebraic","Ingleton-Main condition")

In [159]:
inheritFromMinor("2algebraic",True,principal=True,contraction=False)
#principal extension is currently not required to fill the encyclopedia.

In [160]:
AimpliesB("2algebraic","2flock")

In [161]:
#Flock-representability is closed under circuit-hyperplane relaxation.
closeCHPrelaxation("2flock",True)

Examples:

In [162]:
inspect("2algebraic",(4,9))

{'-': 9572, 'False': 31820, 'True': 148822}

In [163]:
inspect("2algebraic certificate",(4,9))

{'-': 9572,
 '2flock': 1064,
 '2linear': 125692,
 'GF(4)(F)-representable': 98,
 'Ingleton-Main condition': 30756,
 'Quadratic imaginary number field representable': 284,
 'Quaternion representable': 2279,
 'rational': 20469}

In [164]:
inspect("2algebraic",(5,9))

{'-': 12129, 'False': 29263, 'True': 148822}

In [165]:
inspect("2algebraic certificate",(5,9))

{'-': 12129,
 '2flock': 1763,
 '2linear': 125692,
 'GF(4)(F)-representable': 98,
 'Ingleton-Main condition': 27500,
 'Quadratic imaginary number field representable': 284,
 'Quaternion representable': 2279,
 'rational': 20469}

In [166]:
inspect("2algebraic",(3,9))

{'-': 3, 'False': 1, 'True': 1271}

In [167]:
inspect("2algebraic",(6,9))

{'-': 3, 'False': 1, 'True': 1271}

In [168]:
inspect("2algebraic",(4,8))

{'-': 3, 'False': 40, 'True': 897}

In [169]:
#Find the indices of the matroids for which algebraicity is unknown [type (4,8)]
inspect("2algebraic",(4,8),onlySize=False)['-']

[133, 731, 735]

In [170]:
#Check an encyclopedia entry
M[4,8][133]

{'2flock': True,
 '2flock certificate': ['circuit-hyperplane relaxation', 902],
 '2linear': False,
 '2linear certificate': 'Dress-Wenzel condition char 2',
 'Dress-Wenzel condition': False,
 'Dress-Wenzel condition char 2': False,
 'Quadratic imaginary number field representable': False,
 'Quadratic imaginary number field representable certificate': 'Dress-Wenzel condition',
 'circuit-hyperplane relaxations': {11, 731, 732, 734, 735},
 'circuit-hyperplane relaxed': {902},
 'dual': 133,
 'matroid': Matroid of rank 4 on 8 elements with 61 bases,
 'minors r': {5, 28, 95},
 'minors r-1': {65, 69, 103},
 'principal minors r': set(),
 'principal minors r-1': set(),
 'rational': False,
 'rational certificate': 'Dress-Wenzel condition'}

In [171]:
inspect("Dress-Wenzel condition",(4,9))

{'-': 0, 'False': 37988, 'True': 152226}

In [172]:
inspect("Dress-Wenzel condition char 2",(4,9))

{'-': 0, 'False': 64395, 'True': 125819}

In [173]:
inspect("Ingleton-Main condition",(4,9))

{'-': 10636, 'False': 30756, 'True': 148822}

In [174]:
inspect("Ingleton-Main condition depth",(4,9))

{'-': 186595, '3': 1281, '4': 1753, '5': 487, '6': 98}

In [175]:
inspect("Ingleton-Main condition",(5,9))

{'-': 13892, 'False': 27500, 'True': 148822}

In [176]:
inspect("Ingleton-Main condition depth",(5,9))

{'-': 189851, '3': 7, '4': 298, '5': 58}

In [177]:
inspect("2flock",(3,9))

{'-': 0, 'False': 1, 'True': 1274}

In [178]:
inspect("2flock",(4,8))

{'-': 0, 'False': 2, 'True': 938}

In [179]:
inspect("2flock",(4,9))

{'-': 317, 'False': 4551, 'True': 185346}

In [180]:
inspect("2flock",(5,9))

{'-': 315, 'False': 4551, 'True': 185348}